In [1]:
import pandas as pd
from config import CONFIGS
import os
import gc
import boto3

from utils.processing_functions import load_file_local_first, save_file_local_first

IS_LOCAL: True

Checking for local config file and evaluating for updates from S3.
Loading config from local


In [2]:
ENVIRONMENT = os.environ.get("ENVIRONMENT", "dev")
S3_SCRAPER_BUCKET = CONFIGS["s3_scraper_bucket"]
GAME_CONFIGS = CONFIGS["games"]
RATINGS_CONFIGS = CONFIGS["ratings"]
IS_LOCAL = True if os.environ.get("IS_LOCAL", "False").lower() == "true" else False

In [4]:
number_to_retrieve = 5000

In [5]:
def load_reduced_game_df():
        print(f"\nLoading game data from {GAME_CONFIGS['clean_dfs_directory']}")
        game_df = load_file_local_first(
            path=GAME_CONFIGS["clean_dfs_directory"], file_name="games_clean.pkl"
        )

        game_df_reduced = game_df.sort_values("BayesAvgRating", ascending=False)[
            0 : number_to_retrieve
        ]

        del game_df
        gc.collect()
        print("Loaded and refined games data")

        return game_df_reduced

In [6]:
df = load_reduced_game_df()


Loading game data from games/game_dfs_clean
Loaded and refined games data


In [7]:
df.shape

(5000, 47)

In [8]:
df.head()

,BGGId,Name,Description,ImagePath,NumAlternates,NumExpansions,NumImplementations,IsReimplementation,Rank:boardgame,BestPlayers,...,Rank:childrensgames,Rank:cgs,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens
10174,224517,Brass: Birmingham,Brass: Birmingham is an economic strategy game...,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,6,0,1,1,1,3,...,28017,28017,0,1,0,0,0,0,0,0
4846,161936,Pandemic Legacy: Season 1,Pandemic Legacy is a co-operative campaign gam...,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6K...,11,0,2,1,2,4,...,28017,28017,1,1,0,0,0,0,0,0
5989,174430,Gloomhaven,Gloomhaven is a game of Euro-inspired tactica...,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZ...,6,17,1,0,3,3,...,28017,28017,1,1,0,0,0,0,0,0
19051,342942,Ark Nova,"In Ark Nova, you will plan and design a modern...",https://cf.geekdo-images.com/SoU8p28Sk1s8MSvoM...,8,4,0,0,4,2,...,28017,28017,0,1,0,0,0,0,0,0
20469,363622,The Castles of Burgundy: Special Edition,Castles of Burgundy is a legendary Board Game ...,https://cf.geekdo-images.com/JUrmY8GgFPQlENiPT...,2,0,0,0,<NA>,2,...,28017,28017,0,1,0,0,0,0,0,0


In [9]:
game_ids = df["BGGId"].values

In [11]:
dynamodb_client = boto3.client("dynamodb")
item = dynamodb_client.get_item(
                TableName="game_generated_descriptions", Key={"game_id": {"S": "224517"}}
            )["Item"]

In [12]:
for game in game_ids:
    print(f"Getting data for game {game}")

    dynamodb_id = str(game)
    try:
        item = dynamodb_client.get_item(
            TableName="game_generated_descriptions", Key={"game_id": {"S": dynamodb_id}}
        )["Item"]
        df.loc[df['BGGId'] == game, 'generated_pros'] = item['generated_pros']['S']
        df.loc[df['BGGId'] == game, 'generated_cons'] = item['generated_cons']['S']
        df.loc[df['BGGId'] == game, 'generated_description'] = item['generated_description']['S']
    except Exception as e:
        print(f"Game {dynamodb_id} not found in dynamodb, Error: {e}")


Getting data for game 224517
Getting data for game 161936
Getting data for game 174430
Getting data for game 342942
Getting data for game 363622
Getting data for game 233078
Getting data for game 316554
Getting data for game 167791
Getting data for game 115746
Getting data for game 187645
Getting data for game 291457
Getting data for game 162886
Getting data for game 220308
Getting data for game 12333
Getting data for game 182028
Getting data for game 84876
Getting data for game 193738
Getting data for game 169786
Getting data for game 246900
Getting data for game 173346
Getting data for game 28720
Getting data for game 248562
Getting data for game 167355
Getting data for game 271320
Getting data for game 266507
Getting data for game 177736
Getting data for game 124361
Getting data for game 397598
Getting data for game 295770
Getting data for game 341169
Getting data for game 205637
Getting data for game 312484
Getting data for game 120677
Getting data for game 237182
Getting data for 

In [13]:
df.head()

,BGGId,Name,Description,ImagePath,NumAlternates,NumExpansions,NumImplementations,IsReimplementation,Rank:boardgame,BestPlayers,...,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens,generated_pros,generated_cons,generated_description
10174,224517,Brass: Birmingham,Brass: Birmingham is an economic strategy game...,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,6,0,1,1,1,3,...,1,0,0,0,0,0,0,Strategic Depth: Many players appreciate the ...,Steep Learning Curve: Some players find the r...,This game is a strategic economic simulation s...
4846,161936,Pandemic Legacy: Season 1,Pandemic Legacy is a co-operative campaign gam...,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6K...,11,0,2,1,2,4,...,1,0,0,0,0,0,0,Engaging Storyline: The evolving narrative ke...,Complexity: Some players find the numerous ch...,This game is a cooperative board game where pl...
5989,174430,Gloomhaven,Gloomhaven is a game of Euro-inspired tactica...,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZ...,6,17,1,0,3,3,...,1,0,0,0,0,0,0,Content Variety: The game offers an immense a...,Setup Time: Many reviews mention that the gam...,This game is a cooperative tactical adventure ...
19051,342942,Ark Nova,"In Ark Nova, you will plan and design a modern...",https://cf.geekdo-images.com/SoU8p28Sk1s8MSvoM...,8,4,0,0,4,2,...,1,0,0,0,0,0,0,Engaging Theme: The zoo management theme is a...,Lengthy Gameplay: Many reviews highlight that...,This game is a strategic board game where play...
20469,363622,The Castles of Burgundy: Special Edition,Castles of Burgundy is a legendary Board Game ...,https://cf.geekdo-images.com/JUrmY8GgFPQlENiPT...,2,0,0,0,<NA>,2,...,1,0,0,0,0,0,0,High Production Quality: The game boasts exce...,Complexity for New Players: Some reviews ment...,This game is a strategic board game that revol...


In [16]:
df[df['generated_cons'].isna()]

,BGGId,Name,Description,ImagePath,NumAlternates,NumExpansions,NumImplementations,IsReimplementation,Rank:boardgame,BestPlayers,...,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens,generated_pros,generated_cons,generated_description
7831,194517,Super Fantasy Brawl,"In the land of Fabulosa, powerful magics have ...",https://cf.geekdo-images.com/yT5dXJzCpgvBBmXr_...,3,6,1,0,1132,2,...,1,1,0,0,0,0,0,NaN,NaN,NaN
4027,153065,Linko,"In Linko! (a.k.a. Abluxxen), you take turns pl...",https://cf.geekdo-images.com/93dmjdpiUd5J3Vf95...,9,3,0,0,1133,4,...,0,0,1,0,0,0,0,NaN,NaN,NaN
12508,253284,Ticket to Ride: New York,Ticket to Ride: New York features the familiar...,https://cf.geekdo-images.com/v0YHI6x4g8dJdtSqa...,7,1,1,1,1134,2,...,0,0,1,0,0,0,0,NaN,NaN,NaN
26681,823,The Lord of the Rings,The Lord of the Rings is a co-operative game i...,https://cf.geekdo-images.com/oWttjfs235Zh7jLGW...,18,4,0,0,1135,4,...,0,0,0,0,0,0,0,NaN,NaN,NaN
6588,180680,Automobiles,"Drivers, start your engines! Will you cross th...",https://cf.geekdo-images.com/nkJTfSPcFJMWRXbyl...,0,2,0,0,1136,4,...,1,0,1,0,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1362,1212,Nuclear Proliferation,The second expansion set for the Nuclear War c...,https://cf.geekdo-images.com/VXVoVjAcuX40nVTJp...,0,6,0,0,4833,4,...,0,1,0,0,0,0,0,NaN,NaN,NaN
12220,249410,U.S. Telegraph,"U.S. Telegraph, first released as Attika, is s...",https://cf.geekdo-images.com/JawH7MCWCK8e7k36y...,0,0,1,1,4835,2,...,1,0,0,0,0,0,0,NaN,NaN,NaN
12492,252997,Mapmaker: The Gerrymandering Game,"You are a mapmaker, which means you make maps&...",https://cf.geekdo-images.com/JQGTR0FakZt_ZRXgq...,0,0,0,0,4834,2,...,0,0,0,0,1,0,0,NaN,NaN,NaN
12366,251441,Antinomy,/an-tin-uh-mee/ noun&#10;(A contradiction betw...,https://cf.geekdo-images.com/w2GaFp9XHlHS9AHeU...,1,1,0,0,4836,2,...,0,0,0,0,0,0,0,NaN,NaN,NaN


In [14]:
save_file_local_first(path=GAME_CONFIGS["clean_dfs_directory"], file_name=f"top_{number_to_retrieve}_with_attached_rag.pkl", data=df)

data/test/games/game_dfs_clean/top_5000_with_attached_rag.pkl
Saving top_5000_with_attached_rag.pkl to local
Saving top_5000_with_attached_rag.pkl to S3
